# Masterclass Qiskit — 04 • Quantum Machine Learning (QSVM b.a.-ba)

Objectifs (≈90–120 min) :
- Comprendre les briques : embeddings de données, kernels quantiques, modèle SVM.
- Construire un pipeline simple : génération de données 2D, feature map → kernel → SVM.
- Évaluer (accuracy), visualiser la frontière de décision.

**NB** : On se concentre ici sur la logique. Les accélérations matérielles viennent ensuite.

## 1. Données jouets (2 classes en 2D)

In [1]:
import numpy as np
rng = np.random.default_rng(42)

def make_toy(n=40):
    A = rng.normal(loc=[-1.0, -0.5], scale=0.6, size=(n,2))
    B = rng.normal(loc=[+1.0, +0.5], scale=0.6, size=(n,2))
    X = np.vstack([A,B])
    y = np.array([0]*n + [1]*n)
    return X, y

X, y = make_toy(50)
X[:5], y[:10]

(array([[-0.81716975, -1.12399046],
        [-0.54972928,  0.06433883],
        [-2.17062111, -1.2813077 ],
        [-0.92329576, -0.68974556],
        [-1.01008069, -1.01182636]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

## 2. Feature map quantique (angles de rotation)

In [2]:
from qiskit import QuantumCircuit
def feature_map(x):
    # x est un vecteur (x0, x1)
    qc = QuantumCircuit(2)
    qc.h([0,1])
    qc.ry(x[0], 0)
    qc.rx(x[1], 1)
    qc.cx(0,1)
    return qc

## 3. Kernel quantique par "state overlap"

In [3]:
from qiskit.quantum_info import Statevector

def kernel_q(XA, XB):
    K = np.zeros((len(XA), len(XB)))
    for i,a in enumerate(XA):
        psi_a = Statevector.from_instruction(feature_map(a))
        for j,b in enumerate(XB):
            psi_b = Statevector.from_instruction(feature_map(b))
            K[i,j] = np.abs(psi_a.inner(psi_b))**2
    return K

## 4. Entraînement d'un SVM (scikit-learn) avec noyau pré‑calculé

In [4]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)
Ktr = kernel_q(Xtr, Xtr)
Kte = kernel_q(Xte, Xtr)

clf = SVC(kernel='precomputed', C=1.0)
clf.fit(Ktr, ytr)
pred = clf.predict(Kte)
acc = accuracy_score(yte, pred)
acc

1.0

## 5. Visualiser la frontière de décision (maillage 2D)

In [ ]:
import matplotlib.pyplot as plt

def decision_grid(model, Xtr):
    gx, gy = np.meshgrid(np.linspace(X[:,0].min()-1, X[:,0].max()+1, 80),
                         np.linspace(X[:,1].min()-1, X[:,1].max()+1, 80))
    grid = np.c_[gx.ravel(), gy.ravel()]
    Kg = kernel_q(grid, Xtr)
    zz = model.predict(Kg).reshape(gx.shape)
    return gx, gy, zz

gx, gy, zz = decision_grid(clf, Xtr)

plt.figure()
plt.contourf(gx, gy, zz, alpha=0.2)
plt.scatter(Xtr[:,0], Xtr[:,1], c=ytr, marker='o', label='train')
plt.scatter(Xte[:,0], Xte[:,1], c=yte, marker='x', label='test')
plt.legend()
plt.title(f"QSVM kernel overlap — accuracy={acc:.2f}")
plt.show()

## 6. Exercices guidés
1. Changez la distribution des données (`make_toy`) et observez l'évolution.
2. Comparez avec un SVM **classique** (kernel RBF) sur les mêmes données.

In [ ]:
# Zone de réponse — SVM classique pour comparaison
from sklearn.svm import SVC
svm_rbf = SVC(kernel='rbf', gamma='scale', C=1.0).fit(Xtr, ytr)
acc_rbf = accuracy_score(yte, svm_rbf.predict(Xte))
acc, acc_rbf